In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import random
import numpy as np
import copy
from torch.distributions import Categorical


In [14]:
#Hyperparameters
learning_rate = 0.0005
gamma         = 0.98
lmbda         = 0.95
eps_clip      = 0.2
K_epoch       = 5
T_horizon     = 20


In [15]:
class enva(): # player를 9로 해보자!
    def __init__(self):
        self.height = 10
        self.width = 10
        self.num_obstacle_range =5
        self.num_obstacle_range_min = 3
        self.turn = 0
        self.position = int(round(self.width/2))
        self.map = np.array([[0.0 for j in range(self.width)] for i in range(self.height)])
        self.map[self.height-1][self.position] = 2. # 2.
        self.done = False
        
    def step(self, key):
        #obstacle down
        i = self.height - 2
        while(i>=0):
            self.map[i+1] = self.map[i]
            i -= 1
        #obstacle init(first_line)
        self.map[0] = np.array([0 for j in range(self.width)])
        #obstacle making
        num_obstacle = random.randrange(self.num_obstacle_range_min,self.num_obstacle_range)
        i = 0
        while(i < num_obstacle):
            position_obstacle = random.randrange(0,self.width)
            if (self.map[0][position_obstacle] == 1.0):
                continue
            self.map[0][position_obstacle] = 1.0
            i += 1
         #big_obstacle
        big_obstacle = random.randrange(0,20)
        if(big_obstacle == -3):############### 1로 바꾸면 큰 똥이 생긴다.
            big_obstacle = random.randrange(1,self.width-1)
            for j in range(3):
                for k in range(-1,2):
                    self.map[j][big_obstacle + k] = 1.
        #player position
        reward = 1
        # 0 : 왼쪽, 1 : 가만히, 2 : 오른쪽
        if key == 0:
            if(self.position>0):
                self.position -= 1
            
        if key == 1:
            pass      
        if key == 2:
            if(self.position<self.width-1):
                self.position += 1
                  
        if (self.map[self.height-1][self.position] == 1.0):
                    reward = 0
                    self.done = True
                    #print("====Game Over====")
                        
        self.map[self.height-1][self.position] =  2. # 2.
        #reward(turn(time))
        self.turn += 1
        
        return torch.flatten(torch.tensor(self.map),0).numpy(), reward, self.done, _ 

    def reset(self):
        self.turn = 0
        self.position = round(self.width/2)
        self.map = np.array([[0.0 for j in range(self.width)] for i in range(self.height)])
        self.map[self.height-1][self.position] = 2.0
        self.done = False
        
        return torch.flatten(torch.tensor(self.map),0).numpy()

In [16]:
# PPO LSTM 1 by 100 --> good !!!

class PPO(nn.Module):
    def __init__(self):
        super(PPO, self).__init__()
        self.data = []

        self.fc0   = nn.Linear(100,128)
        self.rc1   = nn.LSTM(input_size = 128, hidden_size = 128, num_layers = 2, batch_first=True) # 
        self.fc1   = nn.Linear(128,128)
        self.fc2   = nn.Linear(128,128)

        self.fc4   = nn.Linear(128,128)
        self.fc_pi = nn.Linear(128,3)
        self.fc_v  = nn.Linear(128,1)
        self.optimizer = optim.Adam(self.parameters(), lr=learning_rate)

    def pi(self,x ,h0):
        x = x.view(-1,1,100)  
        x = F.relu(self.fc0(x))
        x, hidden_state = self.rc1(x,h0)
        x = F.relu(self.fc1(x))
        #x = F.relu(self.fc2(x))
        #x = F.relu(self.fc4(x))
        x = self.fc_pi(x)
        prob = F.softmax(x, dim = 2)

        #torch.nan_to_num(prob, nan = 0.0)
        return prob, hidden_state
    
    def v(self, x, h0):
        x = x.view(-1,1,100)
        x = F.relu(self.fc0(x))
        x,_ = self.rc1(x,h0)
        x = F.relu(self.fc1(x))
        #x = F.relu(self.fc2(x))
        #x = F.relu(self.fc4(x))
        v = self.fc_v(x)
        
        return v
      
    def put_data(self, transition):
        self.data.append(transition)
        
    def make_batch(self):
        s_lst, a_lst, r_lst, s_prime_lst, prob_a_lst, h_in_lst, c_in_lst, h_out_lst, c_out_lst, done_lst = [], [], [], [], [], [], [], [], [], []

        for transition in self.data:
            s, a, r, s_prime, prob_a, (h_in,c_in), (h_out,c_out), done = transition
            
            s_lst.append(s)

            a_lst.append([a])
            r_lst.append([r])
            s_prime_lst.append(s_prime)
            prob_a_lst.append([prob_a])
            h_in_lst.append(h_in.detach().numpy())
            c_in_lst.append(c_in.detach().numpy())
            h_out_lst.append(h_out.detach().numpy())
            c_out_lst.append(c_out.detach().numpy())

            done_mask = 0 if done else 1
            done_lst.append([done_mask])
        
        s,a,r,s_prime,done_mask, prob_a = torch.tensor(s_lst, dtype=torch.float), torch.tensor(a_lst), \
                                          torch.tensor(r_lst), torch.tensor(s_prime_lst, dtype=torch.float), \
                                          torch.tensor(done_lst, dtype=torch.float), torch.tensor(prob_a_lst)
        self.data = []
        
        
        h_in_lst = torch.tensor(h_in_lst,dtype = torch.float) 
        h_out_lst = torch.tensor(h_out_lst, dtype = torch.float)
        c_in_lst = torch.tensor(c_in_lst,dtype = torch.float)
        c_out_lst = torch.tensor(c_out_lst, dtype = torch.float)
        
        h_in_lst = h_in_lst.squeeze(2)
        h_out_lst = h_out_lst.squeeze(2)
        c_in_lst = c_in_lst.squeeze(2)
        c_out_lst = c_out_lst.squeeze(2)

        h_in_lst = h_in_lst.permute(1,0,2)
        h_out_lst = h_out_lst.permute(1,0,2)
        c_in_lst = c_in_lst.permute(1,0,2)
        c_out_lst = c_out_lst.permute(1,0,2)
        
        # num_layer X batch X hidden으로 만들기
        return s, a, r, s_prime, done_mask, prob_a, h_in_lst,c_in_lst, h_out_lst,c_out_lst
        
    def train_net(self):
        s, a, r, s_prime, done_mask, prob_a, h1_in,c1_in, h2_out,c2_out = self.make_batch()

        h1 = (h1_in.detach(),c1_in.detach())
        h2 = (h2_out.detach(),c2_out.detach())
        
        
        td_target = r + gamma * self.v(s_prime, h2).squeeze(1) * done_mask
        delta = td_target - self.v(s, h1).squeeze(1)
        delta = delta.detach().numpy()

        advantage_lst = []
        advantage = 0.0
        for delta_t in delta[::-1]:
            advantage = gamma * lmbda * advantage + delta_t[0]
            advantage_lst.append([advantage])
        advantage_lst.reverse()
        advantage = torch.tensor(advantage_lst, dtype=torch.float) 
       
        for i in range(K_epoch):
        
            pi, _ = self.pi(s, h1) 
            pi_a = pi.squeeze(1).gather(1,a) 

            ratio = torch.exp(torch.log(pi_a) - torch.log(prob_a))  # a/b == exp(log(a)-log(b)) ##############

            
            surr1 = ratio * advantage
            surr2 = torch.clamp(ratio, 1-eps_clip, 1+eps_clip) * advantage
            
            loss = -torch.min(surr1, surr2) + F.smooth_l1_loss(self.v(s, h1).squeeze(1) , td_target.detach()) 
            
            self.optimizer.zero_grad()
            loss.mean().backward() # 여러 번 loss의 gradient를 구할 때
            self.optimizer.step()
        
def main():
    hidden = 128
    env = enva()
    model = PPO()
    #model.load_state_dict(torch.load('ppo_ddong_with_dongmin.pt'))

    score = 0.0
    print_interval = 500
    best_score = 0.0
    one_score = 0.0
    
    for n_epi in range(1000000):
        s = env.reset()
        done = False
        h_out = (torch.zeros([2,1,hidden],dtype = torch.float),torch.zeros([2,1,hidden],dtype = torch.float)) #num_layers X batch X hidden size
        while not done:
            for t in range(T_horizon):
                h_in = h_out
                prob, h_out = model.pi(torch.from_numpy(s).float(), h_in)
                prob = prob.view(-1) 
                m = Categorical(prob)
                a = m.sample().item()
                s_prime, r, done, info = env.step(a)
                
                model.put_data((s, a, r, s_prime, prob[a].item(), h_in, h_out, done))
                s = copy.deepcopy(s_prime)
                
                score += r
                one_score += r
                if done:
                    break
            
            model.train_net()
            
        if best_score <= one_score:
            best_score = one_score
            print("... save model ...")
            print(f"best score : {one_score}")
            torch.save(model.state_dict(),'ppo_ddong_with_dongmin.pt')
            
        one_score = 0
            
        if n_epi%print_interval==0 and n_epi!=0:
            print("# of episode :{}, avg score : {:.1f}".format(n_epi, score/print_interval))
            score = 0.0
    
    return best_score

if __name__ == '__main__':
    best_score = main()
    print(f"final best_score : {best_score}")
    


... save model ...
best score : 9.0
... save model ...
best score : 10
... save model ...
best score : 11
... save model ...
best score : 11
... save model ...
best score : 12
... save model ...
best score : 14
... save model ...
best score : 16
... save model ...
best score : 18
... save model ...
best score : 20
... save model ...
best score : 20
... save model ...
best score : 20
# of episode :500, avg score : 10.9
... save model ...
best score : 28
... save model ...
best score : 32
# of episode :1000, avg score : 11.2
... save model ...
best score : 47
# of episode :1500, avg score : 11.9
# of episode :2000, avg score : 11.2
# of episode :2500, avg score : 13.0
# of episode :3000, avg score : 11.6
# of episode :3500, avg score : 11.4
# of episode :4000, avg score : 11.0
# of episode :4500, avg score : 11.2
# of episode :5000, avg score : 11.2
# of episode :5500, avg score : 11.3
# of episode :6000, avg score : 11.2
# of episode :6500, avg score : 13.2
... save model ...
best score

# of episode :93500, avg score : 26.5
# of episode :94000, avg score : 33.5
# of episode :94500, avg score : 33.7
# of episode :95000, avg score : 34.0
# of episode :95500, avg score : 30.0
# of episode :96000, avg score : 32.5
# of episode :96500, avg score : 24.9
# of episode :97000, avg score : 33.2
# of episode :97500, avg score : 35.6
# of episode :98000, avg score : 33.5
# of episode :98500, avg score : 34.4
# of episode :99000, avg score : 26.7
# of episode :99500, avg score : 34.3
# of episode :100000, avg score : 34.3
# of episode :100500, avg score : 33.4
# of episode :101000, avg score : 32.1
# of episode :101500, avg score : 36.0
# of episode :102000, avg score : 29.4
# of episode :102500, avg score : 36.0
# of episode :103000, avg score : 30.3
# of episode :103500, avg score : 23.0
# of episode :104000, avg score : 28.4
# of episode :104500, avg score : 33.4
# of episode :105000, avg score : 33.8
# of episode :105500, avg score : 32.7
# of episode :106000, avg score : 35.8

# of episode :196000, avg score : 50.8
# of episode :196500, avg score : 54.8
# of episode :197000, avg score : 55.8
# of episode :197500, avg score : 52.6
# of episode :198000, avg score : 58.4
# of episode :198500, avg score : 56.0
# of episode :199000, avg score : 56.4
# of episode :199500, avg score : 53.8
# of episode :200000, avg score : 55.5
# of episode :200500, avg score : 54.6
# of episode :201000, avg score : 53.5
# of episode :201500, avg score : 52.2
# of episode :202000, avg score : 51.7
# of episode :202500, avg score : 54.0
# of episode :203000, avg score : 55.7
# of episode :203500, avg score : 49.9
# of episode :204000, avg score : 57.4
# of episode :204500, avg score : 57.8
# of episode :205000, avg score : 59.6
# of episode :205500, avg score : 52.9
# of episode :206000, avg score : 47.8
# of episode :206500, avg score : 50.9
# of episode :207000, avg score : 48.4
# of episode :207500, avg score : 47.6
# of episode :208000, avg score : 37.9
# of episode :208500, avg

# of episode :301000, avg score : 49.4
# of episode :301500, avg score : 43.4
# of episode :302000, avg score : 44.1
# of episode :302500, avg score : 57.9
# of episode :303000, avg score : 55.8
# of episode :303500, avg score : 51.5
# of episode :304000, avg score : 56.0
# of episode :304500, avg score : 59.5
# of episode :305000, avg score : 58.9
# of episode :305500, avg score : 52.8
# of episode :306000, avg score : 58.3
# of episode :306500, avg score : 56.5
# of episode :307000, avg score : 55.0
# of episode :307500, avg score : 55.6
# of episode :308000, avg score : 47.2
# of episode :308500, avg score : 62.4
# of episode :309000, avg score : 58.5
# of episode :309500, avg score : 63.8
# of episode :310000, avg score : 51.9
# of episode :310500, avg score : 58.9
# of episode :311000, avg score : 54.1
# of episode :311500, avg score : 57.8
# of episode :312000, avg score : 56.9
# of episode :312500, avg score : 62.4
# of episode :313000, avg score : 59.4
# of episode :313500, avg

RuntimeError: invalid multinomial distribution (encountering probability entry < 0)

In [ ]:
# PPO LSTM 3 by 100, hidden init

class PPO(nn.Module):
    def __init__(self):
        super(PPO, self).__init__()
        self.data = []

        self.fc0   = nn.Linear(100,128)
        self.rc1   = nn.LSTM(input_size = 128, hidden_size = 128, num_layers = 2, batch_first=True) # 
        self.fc1   = nn.Linear(128*3,128)
        self.fc_pi = nn.Linear(128,3)
        self.fc_v  = nn.Linear(128,1)
        self.optimizer = optim.Adam(self.parameters(), lr=learning_rate)

    def pi(self,x):
        x = x.view(-1,3,100)  
        size = x.size()[0]
        h0 = (torch.zeros([2,size,128],dtype = torch.float),torch.zeros([2,size,128],dtype = torch.float))
        x = F.relu(self.fc0(x))
        x, _ = self.rc1(x,h0)
        x = x.reshape(-1,1,3*128)
        x = F.relu(self.fc1(x))
        x = self.fc_pi(x)
        prob = F.softmax(x, dim = 2)

        torch.nan_to_num(prob, nan = 0.0)
        return prob
    
    def v(self, x):
        x = x.view(-1,3,100)
        size = x.size()[0]
        h0 = (torch.zeros([2,size,128],dtype = torch.float),torch.zeros([2,size,128],dtype = torch.float))
        x = F.relu(self.fc0(x))
        x,_ = self.rc1(x,h0)
        x = x.reshape(-1,1,3*128) # x = x[:,-1,:]
        x = F.relu(self.fc1(x))
        v = self.fc_v(x)
        
        return v
      
    def put_data(self, transition):
        self.data.append(transition)
        
    def make_batch(self):
        s_lst, a_lst, r_lst, s_prime_lst, prob_a_lst, done_lst = [], [], [], [], [], []

        for transition in self.data:
            s, a, r, s_prime, prob_a, done = transition
            
            s_lst.append(s)

            a_lst.append([a])
            r_lst.append([r])
            s_prime_lst.append(s_prime)
            prob_a_lst.append([prob_a])
            done_mask = 0 if done else 1
            done_lst.append([done_mask])
        
        s,a,r,s_prime,done_mask, prob_a = torch.tensor(s_lst, dtype=torch.float), torch.tensor(a_lst), \
                                          torch.tensor(r_lst), torch.tensor(s_prime_lst, dtype=torch.float), \
                                          torch.tensor(done_lst, dtype=torch.float), torch.tensor(prob_a_lst)
        self.data = []
        # num_layer X batch X hidden으로 만들기
        return s, a, r, s_prime, done_mask, prob_a
        
    def train_net(self):
        s, a, r, s_prime, done_mask, prob_a = self.make_batch()

        
        
        td_target = r + gamma * self.v(s_prime).squeeze(1) * done_mask
        delta = td_target - self.v(s).squeeze(1)
        delta = delta.detach().numpy()

        advantage_lst = []
        advantage = 0.0
        for delta_t in delta[::-1]:
            advantage = gamma * lmbda * advantage + delta_t[0]
            advantage_lst.append([advantage])
        advantage_lst.reverse()
        advantage = torch.tensor(advantage_lst, dtype=torch.float) 
       
        for i in range(K_epoch):
        
            pi = self.pi(s) 
            pi_a = pi.squeeze(1).gather(1,a) 

            ratio = torch.exp(torch.log(pi_a) - torch.log(prob_a))  # a/b == exp(log(a)-log(b)) ##############

            
            surr1 = ratio * advantage
            surr2 = torch.clamp(ratio, 1-eps_clip, 1+eps_clip) * advantage
            
            loss = -torch.min(surr1, surr2) + F.smooth_l1_loss(self.v(s).squeeze(1) , td_target.detach()) 
            
            self.optimizer.zero_grad()
            loss.mean().backward() # 여러 번 loss의 gradient를 구할 때
            self.optimizer.step()
        
def main():
    hidden = 128
    env = enva()
    model = PPO()
    #model.load_state_dict(torch.load('ppo_ddong_with_dongmin.pt'))

    score = 0.0
    print_interval = 500
    best_score = 0.0
    one_score = 0.0
    
    for n_epi in range(30000):
        s = env.reset()
        done = False
        s_frame = np.zeros([3,100])
        s_frame[0][95], s_frame[1][95], s_frame[2][95] = 2., 2., 2.

        while not done:
            for t in range(T_horizon):
                prob = model.pi(torch.from_numpy(s_frame).float())
                prob = prob.view(-1) 
                m = Categorical(prob)
                a = m.sample().item()
                s_prime, r, done, info = env.step(a)
                
                current_s_frame = copy.deepcopy(s_frame)
                s_frame[0] = s_frame[1].copy()
                s_frame[1] = s_frame[2].copy()
                s_frame[2] = s_prime.copy()

                model.put_data((current_s_frame, a, r, s_frame, prob[a].item(), done))
                
                score += r
                one_score += r
                if done:
                    break
            
            model.train_net()
            
        if best_score <= one_score:
            best_score = one_score
        #    print("... save model ...")
            print(f"best score : {one_score}")
        #    torch.save(model.state_dict(),'ppo_ddong_with_dongmin.pt')
            
        one_score = 0
            
        if n_epi%print_interval==0 and n_epi!=0:
            print("# of episode :{}, avg score : {:.1f}".format(n_epi, score/print_interval))
            score = 0.0
    
    return best_score

if __name__ == '__main__':
    best_score = main()
    print(f"final best_score : {best_score}")
    
